<a href="https://colab.research.google.com/github/thivi/batting-lineup-generator/blob/master/EC_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deap

     |████████████████████████████████| 163kB 2.8MB/s 


In [ ]:
playersData=[
{"name":"MDKJ Perera","balls":21.5625,"runs":1.38260869565217},
{"name":"BKG Mendis","balls":17.1111111111111,"runs":1.33116883116883},
{"name":"NLTC Perera","balls":19.0909090909091,"runs":1.64761904761905},
{"name":"MD Gunathilaka","balls":17,"runs":1.26470588235294},
{"name":"MD Shanaka","balls":16.3333333333333,"runs":1.16734693877551},
{"name":"I Udana","balls":16.875,"runs":1.5037037037037},
{"name":"WIA Fernando","balls":13.8571428571429,"runs":1.00515463917526},
{"name":"N Dickwella","balls":12.6363636363636,"runs":1.37410071942446},
{"name":"DM de Silva","balls":18.375,"runs":1.1156462585034},
{"name":"PBB Rajapaksa","balls":21.4,"runs":1.30841121495327},
{"name":"PWH de Silva","balls":14.8571428571429,"runs":1.18269230769231},
{"name":"BOP Fernando","balls":21.75,"runs":1.25287356321839},
{"name":"GSNFG Jayasuriya","balls":13.5,"runs":1.03703703703704},
{"name":"PHKD Mendis","balls":11.5,"runs":1.43478260869565},
{"name":"AD Mathews","balls":13.25,"runs":1.20754716981132},
]

In [ ]:
import random

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

ModuleNotFoundError: ignored

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [ ]:
toolbox = base.Toolbox()

In [ ]:
toolbox.register("random_batsman", random.randint, 0, 14)

In [ ]:
toolbox.register("indices", random.sample, range(15), 15)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
def evaluationMaxRuns(individual):
  total_runs=0
  total_balls=0
  total_wickets=0

  while(total_balls<120 and total_wickets<7):
    batsman_strike_rate=playersData[individual[total_wickets]]["runs"]
    batsman_balls_per_innings=round(playersData[individual[total_wickets]]["balls"],0)
    
    if (total_balls + batsman_balls_per_innings > 120):
      batsman_balls_per_innings=120-total_balls

    batsman_runs=round(batsman_balls_per_innings*(batsman_strike_rate),0)
    total_balls+=batsman_balls_per_innings
    total_runs+=batsman_runs
    total_wickets+=1

  return total_runs,

In [ ]:
toolbox.register("evaluate", evaluationMaxRuns)
toolbox.register("crossover", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("parent_selection", tools.selTournament, tournsize=3)

In [ ]:
def main():
  #Generate a population of 300 individuals
  pop=toolbox.population(n=400)

  #Calculate the fitness value of each individual in the population
  fitnesses = list(map(toolbox.evaluate, pop))

  #Assign the fitness value of every individual to the individual's object
  for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

  #probability of crossover and mutation
  CXPB, MUTPB = 0.8, 0.4

  #get the fitness value of individuals into a list
  fits = [ind.fitness.values[0] for ind in pop]

  g=0
  while (g<100):
    offspring = toolbox.parent_selection(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < CXPB:
          toolbox.crossover(child1, child2)
          del child1.fitness.values
          del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    pop[:] = offspring

    fits = [ind.fitness.values[0] for ind in pop]
        
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)

    g+=1
  
  best_ind = tools.selBest(pop, 1)[0]
  print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
  print("The ideal batting lineup is:")

  order_no=1
  for index in best_ind:
    if order_no>7:
      break
    print(order_no, playersData[index]["name"])
    order_no+=1

In [ ]:
main()

NameError: ignored